In [8]:
import pandas as pd
import numpy as np

df = pd.read_csv('ObesityDataSet_raw_and_data_sinthetic.csv')

df.head()

,Age,Gender,Height,Weight,CALC,FAVC,FCVC,NCP,SCC,SMOKE,CH2O,family_history_with_overweight,FAF,TUE,CAEC,MTRANS,NObeyesdad
0,21.0,Female,1.62,64.0,no,no,2.0,3.0,no,no,2.0,yes,0.0,1.0,Sometimes,Public_Transportation,Normal_Weight
1,21.0,Female,1.52,56.0,Sometimes,no,3.0,3.0,yes,yes,3.0,yes,3.0,0.0,Sometimes,Public_Transportation,Normal_Weight
2,23.0,Male,1.80,77.0,Frequently,no,2.0,3.0,no,no,2.0,yes,2.0,1.0,Sometimes,Public_Transportation,Normal_Weight
3,27.0,Male,1.80,87.0,Frequently,no,3.0,3.0,no,no,2.0,no,2.0,0.0,Sometimes,Walking,Overweight_Level_I
4,22.0,Male,1.78,89.8,Sometimes,no,2.0,1.0,no,no,2.0,no,0.0,0.0,Sometimes,Public_Transportation,Overweight_Level_II


 Target: классы `NObeyesdad`

Мы решаем задачу **классификации**.  
 Значит таргет `NObeyesdad` содержит **категории (классы)**, которые модель должна предсказывать.

1. Узнать, какие классы вообще есть
Это нужно, чтобы понимать, какие варианты ответа возможны.



In [4]:
classes = sorted(df["NObeyesdad"].unique())
classes

['Insufficient_Weight',
 'Normal_Weight',
 'Obesity_Type_I',
 'Obesity_Type_II',
 'Obesity_Type_III',
 'Overweight_Level_I',
 'Overweight_Level_II']

2. Посчитать, сколько примеров каждого класса

Это нужно, чтобы проверить баланс: если какой-то класс встречается редко, метрики и обучение могут смещаться в сторону частых классов.

In [5]:
df["NObeyesdad"].value_counts()

NObeyesdad
Obesity_Type_I         351
Obesity_Type_III       324
Obesity_Type_II        297
Overweight_Level_I     290
Overweight_Level_II    290
Normal_Weight          287
Insufficient_Weight    272
Name: count, dtype: int64

In [6]:
(df["NObeyesdad"].value_counts(normalize=True) * 100).round(2)

NObeyesdad
Obesity_Type_I         16.63
Obesity_Type_III       15.35
Obesity_Type_II        14.07
Overweight_Level_I     13.74
Overweight_Level_II    13.74
Normal_Weight          13.60
Insufficient_Weight    12.88
Name: proportion, dtype: float64

 Количество объектов по классам лежит в диапазоне 272–351, то есть сильного дисбаланса нет.  
Значит можно использовать `accuracy` как baseline, но дополнительно смотреть `macro-F1`, так как классов много и важно качество по каждому.

Типы признаков: числовые vs категориальные

Это нужно, чтобы понять **какой препроцессинг потребуется**:
- числовые → (возможно) scaling
- категориальные → one-hot encoding
- модели: линейные чувствительны к масштабу, деревья — почти нет

In [ ]:
target = "NObeyesdad"
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in df.columns if c not in num_cols and c != target]

print("Numeric:", num_cols)
print("Categorical:", cat_cols)

Numeric: ['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE']
Categorical: ['Gender', 'CALC', 'FAVC', 'SCC', 'SMOKE', 'family_history_with_overweight', 'CAEC', 'MTRANS']


Проверяем пропуски

In [9]:
df.isna().sum().sort_values(ascending=False).head(20)

Age                               0
SMOKE                             0
MTRANS                            0
CAEC                              0
TUE                               0
FAF                               0
family_history_with_overweight    0
CH2O                              0
SCC                               0
Gender                            0
NCP                               0
FCVC                              0
FAVC                              0
CALC                              0
Weight                            0
Height                            0
NObeyesdad                        0
dtype: int64

Смотрим диапазоны и возможные выбросы (min/max/mean/percentiles).

In [10]:
df[num_cols].describe().T

,count,mean,std,min,25%,50%,75%,max
Age,2111.0,24.312600,6.345968,14.00,19.947192,22.777890,26.000000,61.00
Height,2111.0,1.701677,0.093305,1.45,1.630000,1.700499,1.768464,1.98
Weight,2111.0,86.586058,26.191172,39.00,65.473343,83.000000,107.430682,173.00
FCVC,2111.0,2.419043,0.533927,1.00,2.000000,2.385502,3.000000,3.00
NCP,2111.0,2.685628,0.778039,1.00,2.658738,3.000000,3.000000,4.00
CH2O,2111.0,2.008011,0.612953,1.00,1.584812,2.000000,2.477420,3.00
FAF,2111.0,1.010298,0.850592,0.00,0.124505,1.000000,1.666678,3.00
TUE,2111.0,0.657866,0.608927,0.00,0.000000,0.625350,1.000000,2.00


Какие значения принимают категориальные признаки

In [11]:
for c in cat_cols:
    print(f"\n{c}: {df[c].nunique()} unique values")
    display(df[c].value_counts())


Gender: 2 unique values


Gender
Male      1068
Female    1043
Name: count, dtype: int64


CALC: 4 unique values


CALC
Sometimes     1401
no             639
Frequently      70
Always           1
Name: count, dtype: int64


FAVC: 2 unique values


...

family_history_with_overweight
yes    1726
no      385
Name: count, dtype: int64


CAEC: 4 unique values


CAEC
Sometimes     1765
Frequently     242
Always          53
no              51
Name: count, dtype: int64


MTRANS: 5 unique values


MTRANS
Public_Transportation    1580
Automobile                457
Walking                    56
Motorbike                  11
Bike                        7
Name: count, dtype: int64

Смотрим связь признаков с таргетом


Сравним **средние значения** числовых признаков по каждому классу таргета.  
Если у признака средние сильно отличаются между классами → это потенциально сильный признак.

In [12]:
mean_by_class = df.groupby(target)[num_cols].mean().round(3)
mean_by_class

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
NObeyesdad,,,,,,,,
Insufficient_Weight,19.783,1.691,49.906,2.481,2.914,1.871,1.250,0.839
Normal_Weight,21.739,1.677,62.155,2.334,2.739,1.850,1.247,0.676
Obesity_Type_I,25.885,1.694,92.870,2.186,2.432,2.112,0.987,0.677
Obesity_Type_II,28.234,1.772,115.305,2.391,2.745,1.878,0.972,0.515
Obesity_Type_III,23.496,1.688,120.941,3.000,3.000,2.208,0.665,0.605
Overweight_Level_I,23.418,1.688,74.267,2.265,2.504,2.059,1.057,0.613
Overweight_Level_II,26.997,1.704,82.085,2.261,2.496,2.025,0.958,0.697


Выводы по числовым признакам (по средним значениям)

Мы посмотрели средние значения числовых признаков в каждом классе `NObeyesdad`.
Что можно понять:

1) **`Weight` (вес) — самый сильный признак**
- Средний вес сильно растёт от `Insufficient_Weight` к `Obesity_Type_III`.
- Это значит, что модель почти точно будет активно использовать `Weight`.
- Следствие: качество может быть высоким даже на простых моделях.

2) **`Height` (рост) меняется мало**
- Средние значения роста почти одинаковые у разных классов.
- Значит один `Height` слабее объясняет классы, но он может быть полезен в комбинации с весом.

3) **`Age` (возраст) растёт, но не так сильно, как вес**
- Средний возраст увеличивается не монотонно идеально, но тренд в сторону более высоких классов заметен.
- Значит `Age` вероятно полезный, но не главный признак.

4) Признаки привычек (`FCVC`, `NCP`, `CH2O`, `FAF`, `TUE`) **меняются слабее**
- Различия по классам есть, но они меньше, чем у `Weight`.
- Это означает, что:
  - либо их вклад реально меньше,
  - либо они важны **в сочетаниях** (например, физическая активность важна при определённом возрасте/питании).

Что это говорит про выбор модели 
- Так как один признак (`Weight`) уже сильно разделяет классы, **линейные модели могут дать хороший результат.
- Но так как поведенческие признаки могут работать во взаимодействиях, **деревья могут дать прирост** за счёт нелинейностей.


Корреляции числовых признаков

In [13]:
corr = df[num_cols].corr()
corr.round(2)

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE
Age,1.00,-0.03,0.20,0.02,-0.04,-0.05,-0.14,-0.30
Height,-0.03,1.00,0.46,-0.04,0.24,0.21,0.29,0.05
Weight,0.20,0.46,1.00,0.22,0.11,0.20,-0.05,-0.07
FCVC,0.02,-0.04,0.22,1.00,0.04,0.07,0.02,-0.10
NCP,-0.04,0.24,0.11,0.04,1.00,0.06,0.13,0.04
CH2O,-0.05,0.21,0.20,0.07,0.06,1.00,0.17,0.01
FAF,-0.14,0.29,-0.05,0.02,0.13,0.17,1.00,0.06
TUE,-0.30,0.05,-0.07,-0.10,0.04,0.01,0.06,1.00


In [14]:
corr_pairs = (
    corr.abs()
    .where(np.triu(np.ones(corr.shape), k=1).astype(bool))
    .stack()
    .sort_values(ascending=False)
)

corr_pairs.head(15)

Height  Weight    0.463136
Age     TUE       0.296931
Height  FAF       0.294709
        NCP       0.243672
Weight  FCVC      0.216125
Height  CH2O      0.213376
Age     Weight    0.202560
Weight  CH2O      0.200575
CH2O    FAF       0.167236
Age     FAF       0.144938
NCP     FAF       0.129504
Weight  NCP       0.107469
FCVC    TUE       0.101135
Weight  TUE       0.071561
FCVC    CH2O      0.068461
dtype: float64

1) Сильных корреляций нет: максимальная |corr| ≈ 0.46 (Height–Weight).  
   Значит сильной мультиколлинеарности нет → для линейных моделей это хорошая новость.

2) Height и Weight умеренно коррелируют (≈ 0.46).  
   Это логично и намекает, что позже стоит проверить гипотезу про новый признак (например BMI), который объединяет рост и вес.

3) Остальные корреляции слабые (≈ 0.30 и ниже).  
   Это значит, что поведенческие признаки (FAF, TUE, CH2O и т.д.) не дублируют друг друга и могут давать дополнительный сигнал.
   

Таблицы долей: категории внутри каждого класса


In [15]:
for c in cat_cols:
    print("\n===", c, "===")
    tab = pd.crosstab(df[c], df[target], normalize="columns")
    display(tab.round(3))


=== Gender ===


NObeyesdad,Insufficient_Weight,Normal_Weight,Obesity_Type_I,Obesity_Type_II,Obesity_Type_III,Overweight_Level_I,Overweight_Level_II
Gender,,,,,,,
Female,0.636,0.491,0.444,0.007,0.997,0.5,0.355
Male,0.364,0.509,0.556,0.993,0.003,0.5,0.645



=== CALC ===


NObeyesdad,Insufficient_Weight,Normal_Weight,Obesity_Type_I,Obesity_Type_II,Obesity_Type_III,Overweight_Level_I,Overweight_Level_II
CALC,,,,,,,
Always,0.000,0.003,0.00,0.000,0.000,0.000,0.000
Frequently,0.004,0.063,0.04,0.007,0.000,0.055,0.066
Sometimes,0.566,0.561,0.49,0.754,0.997,0.772,0.493
no,0.430,0.373,0.47,0.239,0.003,0.172,0.441



=== FAVC ===


...

NObeyesdad,Insufficient_Weight,Normal_Weight,Obesity_Type_I,Obesity_Type_II,Obesity_Type_III,Overweight_Level_I,Overweight_Level_II
family_history_with_overweight,,,,,,,
no,0.537,0.46,0.02,0.003,0.0,0.279,0.062
yes,0.463,0.54,0.98,0.997,1.0,0.721,0.938



=== CAEC ===


NObeyesdad,Insufficient_Weight,Normal_Weight,Obesity_Type_I,Obesity_Type_II,Obesity_Type_III,Overweight_Level_I,Overweight_Level_II
CAEC,,,,,,,
Always,0.007,0.122,0.017,0.007,0.000,0.017,0.010
Frequently,0.445,0.289,0.017,0.003,0.003,0.048,0.055
Sometimes,0.537,0.554,0.963,0.987,0.997,0.814,0.931
no,0.011,0.035,0.003,0.003,0.000,0.121,0.003



=== MTRANS ===


NObeyesdad,Insufficient_Weight,Normal_Weight,Obesity_Type_I,Obesity_Type_II,Obesity_Type_III,Overweight_Level_I,Overweight_Level_II
MTRANS,,,,,,,
Automobile,0.169,0.157,0.313,0.320,0.003,0.228,0.324
Bike,0.000,0.014,0.000,0.003,0.000,0.007,0.000
Motorbike,0.000,0.021,0.009,0.000,0.000,0.003,0.003
Public_Transportation,0.809,0.697,0.672,0.673,0.997,0.731,0.652
Walking,0.022,0.111,0.006,0.003,0.000,0.031,0.021


### Что показал `strength`

- Фичи **вверху списка** сильнее всего отличаются по классам → **они полезные** (после one-hot должны помогать модели).
- Фичи **внизу списка** почти одинаковые во всех классах → **слабее**, но могут помогать в комбинации с другими.

Вывод: категориальные признаки точно **кодируем (OneHotEncoder)**, а на фичи из топа `strength` делаем упор в моделях/гипотезах.

### Гипотеза: добавление ИМТ (BMI) улучшит качество

ИМТ:

$$
BMI = \frac{Weight}{Height^2}
$$

Почему: вес сильно связан с классами, рост сам по себе слабее; ИМТ объединяет их в более “смысловой” признак и может лучше отделять уровни ожирения, особенно соседние.

Как проверить: обучить один и тот же baseline (например, логрегрессия/дерево) 1) без BMI 2) с BMI и сравнить `accuracy` и `macro-F1` на одинаковой валидации.

Ожидаем: `macro-F1` станет выше.

In [17]:
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, f1_score

target = "NObeyesdad"
X = df.drop(columns=[target])
y = df[target]

num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in X.columns if c not in num_cols]

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scoring = {
    "accuracy": "accuracy",
    "f1_macro": make_scorer(f1_score, average="macro")
}

def eval_pipeline(X_used, num_used):
    pre = ColumnTransformer([
        ("num", StandardScaler(), num_used),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ])
    model = LogisticRegression(max_iter=2000)
    pipe = Pipeline([("pre", pre), ("model", model)])
    res = cross_validate(pipe, X_used, y, cv=cv, scoring=scoring)
    return {
        "accuracy_mean": res["test_accuracy"].mean(),
        "accuracy_std": res["test_accuracy"].std(),
        "f1_macro_mean": res["test_f1_macro"].mean(),
        "f1_macro_std": res["test_f1_macro"].std(),
    }

# 1) Без BMI
res_no_bmi = eval_pipeline(X, num_cols)

# 2) С BMI
X_bmi = X.copy()
X_bmi["BMI"] = X_bmi["Weight"] / (X_bmi["Height"] ** 2)
res_bmi = eval_pipeline(X_bmi, num_cols + ["BMI"])

pd.DataFrame([res_no_bmi, res_bmi], index=["no_BMI", "with_BMI"]).round(4)

,accuracy_mean,accuracy_std,f1_macro_mean,f1_macro_std
no_BMI,0.8801,0.0113,0.8762,0.0113
with_BMI,0.9105,0.0134,0.9077,0.0135



Сравнили Logistic Regression (одинаковый пайплайн) в двух вариантах:

- **без BMI:** accuracy = 0.8801, macro-F1 = 0.8762  
- **с BMI:**  accuracy = 0.9105, macro-F1 = 0.9077  

Добавление BMI дало прирост примерно **+0.03** по accuracy и **+0.03** по macro-F1.

**Вывод:** гипотеза подтверждается — BMI действительно добавляет сильный сигнал и улучшает качество модели.

Категориальные признаки содержат полезную информацию, а не просто “шум”.

In [18]:
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, f1_score

target = "NObeyesdad"
RANDOM_STATE = 42

X = df.drop(columns=[target]).copy()
y = df[target].copy()

# добавим BMI (уже проверили, что это полезно)
X["BMI"] = X["Weight"] / (X["Height"] ** 2)

num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in X.columns if c not in num_cols]

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
scoring = {
    "accuracy": "accuracy",
    "f1_macro": make_scorer(f1_score, average="macro"),
}

def make_pipe(use_cats: bool):
    transformers = [("num", StandardScaler(), num_cols)]
    if use_cats:
        transformers.append(("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols))

    pre = ColumnTransformer(transformers=transformers, remainder="drop")
    model = LogisticRegression(max_iter=3000, random_state=RANDOM_STATE)
    return Pipeline([("preprocess", pre), ("model", model)])

def eval_pipe(pipe):
    res = cross_validate(pipe, X, y, cv=cv, scoring=scoring, n_jobs=-1)
    return {
        "accuracy_mean": res["test_accuracy"].mean(),
        "accuracy_std": res["test_accuracy"].std(),
        "f1_macro_mean": res["test_f1_macro"].mean(),
        "f1_macro_std": res["test_f1_macro"].std(),
    }

res_num_only = eval_pipe(make_pipe(use_cats=False))
res_with_cats = eval_pipe(make_pipe(use_cats=True))

pd.DataFrame([res_num_only, res_with_cats], index=["num_only(+BMI)", "num+cat(+BMI)"]).round(4)

,accuracy_mean,accuracy_std,f1_macro_mean,f1_macro_std
num_only(+BMI),0.9147,0.0080,0.9120,0.0076
num+cat(+BMI),0.9105,0.0134,0.9077,0.0135


Ожидали, что добавление категориальных признаков улучшит качество, но получилось наоборот:

- **только числовые (+BMI):** accuracy = 0.9147, macro-F1 = 0.9120  
- **числовые (+BMI) + категориальные:** accuracy = 0.9105, macro-F1 = 0.9077  

Качество **чуть снизилось** после добавления категориальных признаков (разница небольшая, но в минус).

Что это может значить:
1) Основной сигнал уже содержится в `Weight/Height` и особенно в `BMI`, а категориальные дают мало дополнительной информации.
2) One-hot добавляет много признаков → для LogReg это может дать небольшой шум/переобучение.
3) Разница сопоставима со std по фолдам, поэтому эффект слабый: “категориальные не дали явного прироста”.

Итого: можно оставить только числовые + BMI

Гипотеза: дерево решений переобучается, поэтому ограничение глубины улучшит качество

In [19]:
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import make_scorer, f1_score

target = "NObeyesdad"
RANDOM_STATE = 42

X = df.drop(columns=[target]).copy()
y = df[target].copy()

# добавим BMI (мы уже показали, что он улучшает качество)
X["BMI"] = X["Weight"] / (X["Height"] ** 2)

num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in X.columns if c not in num_cols]

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
scoring = {
    "accuracy": "accuracy",
    "f1_macro": make_scorer(f1_score, average="macro"),
}

pre = ColumnTransformer(
    transformers=[
        ("num", "passthrough", num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ],
    remainder="drop",
)

def eval_tree(max_depth):
    model = DecisionTreeClassifier(max_depth=max_depth, random_state=RANDOM_STATE)
    pipe = Pipeline([("preprocess", pre), ("model", model)])
    res = cross_validate(pipe, X, y, cv=cv, scoring=scoring, n_jobs=-1)
    return {
        "max_depth": max_depth,
        "accuracy_mean": res["test_accuracy"].mean(),
        "accuracy_std": res["test_accuracy"].std(),
        "f1_macro_mean": res["test_f1_macro"].mean(),
        "f1_macro_std": res["test_f1_macro"].std(),
    }

depths = [None, 2, 3, 5, 7, 10, 15]
results = [eval_tree(d) for d in depths]

pd.DataFrame(results).round(4).sort_values("f1_macro_mean", ascending=False)

,max_depth,accuracy_mean,accuracy_std,f1_macro_mean,f1_macro_std
4,7.0,0.9749,0.0066,0.9747,0.0069
2,3.0,0.9730,0.0090,0.9729,0.0092
0,NaN,0.9721,0.0081,0.9717,0.0082
5,10.0,0.9721,0.0081,0.9717,0.0082
6,15.0,0.9721,0.0081,0.9717,0.0082
3,5.0,0.9702,0.0080,0.9698,0.0082
1,2.0,0.5846,0.0010,0.4262,0.0017


Гипотеза подтвердилась: ограничение глубины улучшает качество.

Лучший результат получился при `max_depth = 7`:
- accuracy ≈ 0.9749
- macro-F1 ≈ 0.9747

Гипотеза: RandomForest будет лучше одного дерева

In [20]:
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import make_scorer, f1_score

target = "NObeyesdad"
RANDOM_STATE = 42

X = df.drop(columns=[target]).copy()
y = df[target].copy()

# BMI (подтверждённая гипотеза)
X["BMI"] = X["Weight"] / (X["Height"] ** 2)

num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in X.columns if c not in num_cols]

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
scoring = {
    "accuracy": "accuracy",
    "f1_macro": make_scorer(f1_score, average="macro"),
}

pre = ColumnTransformer(
    transformers=[
        ("num", "passthrough", num_cols),
        ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
    ],
    remainder="drop",
)

tree = Pipeline([
    ("preprocess", pre),
    ("model", DecisionTreeClassifier(max_depth=7, random_state=RANDOM_STATE)),
])

rf = Pipeline([
    ("preprocess", pre),
    ("model", RandomForestClassifier(
        n_estimators=300,
        random_state=RANDOM_STATE,
        n_jobs=-1
    )),
])

res_tree = cross_validate(tree, X, y, cv=cv, scoring=scoring, n_jobs=-1)
res_rf = cross_validate(rf, X, y, cv=cv, scoring=scoring, n_jobs=-1)

out = pd.DataFrame([
    {
        "model": "DecisionTree(depth=7)",
        "accuracy_mean": res_tree["test_accuracy"].mean(),
        "accuracy_std": res_tree["test_accuracy"].std(),
        "f1_macro_mean": res_tree["test_f1_macro"].mean(),
        "f1_macro_std": res_tree["test_f1_macro"].std(),
    },
    {
        "model": "RandomForest(300)",
        "accuracy_mean": res_rf["test_accuracy"].mean(),
        "accuracy_std": res_rf["test_accuracy"].std(),
        "f1_macro_mean": res_rf["test_f1_macro"].mean(),
        "f1_macro_std": res_rf["test_f1_macro"].std(),
    },
]).round(4)

out

,model,accuracy_mean,accuracy_std,f1_macro_mean,f1_macro_std
0,DecisionTree(depth=7),0.9749,0.0066,0.9747,0.0069
1,RandomForest(300),0.9820,0.0063,0.9813,0.0063


Гипотеза подтвердилась.

- **DecisionTree(depth=7):** accuracy = 0.9749 (std 0.0066), macro-F1 = 0.9747 (std 0.0069)
- **RandomForest(300):** accuracy = 0.9820 (std 0.0063)

RandomForest показал **более высокую точность** (примерно +0.007 по accuracy) и сопоставимую стабильность (std чуть меньше).

 Гипотеза: class_weight='balanced' почти не изменит качество

In [21]:
from sklearn.model_selection import StratifiedKFold, cross_validate
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import make_scorer, f1_score

target = "NObeyesdad"
RANDOM_STATE = 42

X = df.drop(columns=[target]).copy()
y = df[target].copy()

# используем BMI, так как уже подтвердили, что он полезен
X["BMI"] = X["Weight"] / (X["Height"] ** 2)

num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
cat_cols = [c for c in X.columns if c not in num_cols]

cv = StratifiedKFold(n_splits=5, shuffle=True, random_state=RANDOM_STATE)
scoring = {
    "accuracy": "accuracy",
    "f1_macro": make_scorer(f1_score, average="macro"),
}

def eval_logreg(class_weight=None):
    pre = ColumnTransformer(
        transformers=[
            ("num", StandardScaler(), num_cols),
            ("cat", OneHotEncoder(handle_unknown="ignore"), cat_cols),
        ],
        remainder="drop",
    )
    model = LogisticRegression(max_iter=3000, random_state=RANDOM_STATE, class_weight=class_weight)
    pipe = Pipeline([("preprocess", pre), ("model", model)])
    res = cross_validate(pipe, X, y, cv=cv, scoring=scoring, n_jobs=-1)
    return {
        "accuracy_mean": res["test_accuracy"].mean(),
        "accuracy_std": res["test_accuracy"].std(),
        "f1_macro_mean": res["test_f1_macro"].mean(),
        "f1_macro_std": res["test_f1_macro"].std(),
    }

res_no_cw = eval_logreg(class_weight=None)
res_balanced = eval_logreg(class_weight="balanced")

pd.DataFrame([res_no_cw, res_balanced], index=["no_class_weight", "class_weight_balanced"]).round(4)

,accuracy_mean,accuracy_std,f1_macro_mean,f1_macro_std
no_class_weight,0.9105,0.0134,0.9077,0.0135
class_weight_balanced,0.9109,0.0108,0.9082,0.0110


Гипотеза подтвердилась: эффект от `class_weight='balanced'` практически нулевой.

- **без весов:** accuracy = 0.9105, macro-F1 = 0.9077  
- **с class_weight='balanced':** accuracy = 0.9109, macro-F1 = 0.9082  

Разница очень маленькая и сопоставима со стандартным отклонением по фолдам, то есть это не “реальный прирост”.
